# Introduction

This notebook allows you to run a simplified version of the ANTARESS workflow, for the following purpose:
- format and clean time-series of echelle spectra (S2D) output by standard spectrographs DRS

At the end of the notebook, 

Set `working_path`, `star_name`, and `pl_name` to the same values that you used in the [set-up notebook](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_setup.ipynb) to initialize your mock system.


To try the workflow, place your observational dataset in the directory of your choice (named, for example, `Working_dir/`), and set the `working_path` to the directory containing your data.

Some options and functionallities are not included in this notebook to keep it slim and easy to use. For targets with multiple visists it is espesecially recommended to use the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

Run the cell below to retrieve the system and visits properties defined in the `ANTARESS_nbook_setup.ipynb`.

In [ ]:
import ANTARESS_nbook_bground
from antaress.ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher

setup = {}
setup['working_path'] ='/Users/erikfriden/Desktop/antaress/working_dir/'

setup['par'] = {
    'star_name' : 'TOI421',
    'pl_name'   : 'TOI421c'
}

input_nbook = ANTARESS_nbook_bground.load_nbook(setup, 'sp_reduc')

# Processing of spectral data

<a id='inst_cal'></a>
**Instrumental calibration**

The input spectra in `'data_dir'` are assumed to be corrected for standard nstrumental effects such as flat field, blaze, and background.

- `blaze`: `True` to measure calibration profile and detector noise from blazed data, S2D_BLAZE fits file must be provided in the same directory as S2D data.
- `calc_gcal`: `True` to calculate results and `False` to retrieve previous results.

Run this cell to measure the instrumental calibration. It is used in some modules for the rescaling of photoelectrons to their original photoelectron levels. 

In [ ]:
input_nbook['sp_reduc'].update({'blaze'    : True,
                               'calc_gcal' : False})
ANTARESS_nbook_bground.inst_cal(input_nbook, plot=False)

**Launching the ANTARESS workflow**

Run this cell to launch the ANTARESS workflow. During the spectral reduction it is a good practice to inspect the corrections after each step since the next correction will be applied to spectra after the previous correction step. 

In [ ]:
input_nbook['sp_reduc'].update({'proc_data' : True})
ANTARESS_nbook_bground.processing_mode(input_nbook)

In [ ]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

<a id='process_data'></a>
**ANTARESS in calculation/retrieval mode**

To avoid unecessary computation time we deactivate the reading in of the observed data after this step since all corrections will be appied to the corrected spectra from the previous step. 

However, if pixels are later being masked and excluded from the correction after this step you will have to rerun the previous step in calculation mode again. At this point, the cell for masking of pixels would already have been executed, and you only have to activate and run this cell again. After you can rerun the instrumental calibration by running that cell above and launching the ANTARESS workflow. The instrumental calibration will then be performed on the observational data exluding the masked pixels. Again, deactivate the processing of observaitonal data in this cell and continue to run the cells below and perform the corrections once more but now with the pixels masked. Inspect the new corrections and continue with the workflow.

In [ ]:
input_nbook['sp_reduc'].update({'proc_data' : False})
ANTARESS_nbook_bground.processing_mode(input_nbook)

**Telluric correction**

Run this cell to perform the telluric correction. 

- `tell_species` (list, string): choose which molecules to be used in correction (H$_2$O, O$_2$, CH$_4$, CO, CO$_2$)       
- `tell_thresh` (float): threshold for contrast of telluric lines to be masked as nan



In [ ]:
input_nbook['sp_reduc'].update({
    'tell_species'     : ['H2O', 'O2'],
    'tell_thresh'      : 0.1
})

# Calculate results (True) or retrieve previous results (False)
input_nbook['sp_reduc'].update({'calc_tell' : False})

ANTARESS_nbook_bground.tell_corr(input_nbook, plot=False)

**Adding a spectral plotting option here**

Run this cell to plot either the transmission spectrum `sp_trans` or the raw spectrum `sp_raw` before and after the telluric correction.




In [ ]:
input_nbook['sp_reduc'].update({
    'sp_raw'     : False,
    'trans_sp'   : False,
    'wav_range'  : [3000,7000],
    'y_range'    : None,
    'norm_prof'  : True,
    'plot_master': False,
    'multi_exp'  :True
})

In [ ]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**Masking of pixels**

Run this cell to mask pixels/ranges to exclude from the correction. 

Indicate which spectral orders to be masked and the corresponding wavelength range to mask

- `order` (list): spectral orders to be masked, e.g., [10,24]
- `range` (list): wavelength range to be masked for the spectral orders in the list `order`. The list must have the same length as the list in `order`, the format as $[[\lambda_{\mathrm{low}, \mathrm{ord}_{10}}, \lambda_{\mathrm{high}, \mathrm{ord}_{10}}], [\lambda_{\mathrm{low}, \mathrm{ord}_{24}}, \lambda_{\mathrm{high}, \mathrm{ord}_{24}}]]$, for the ranges to be masked in the spectral orders defined in `order`.

If you chosed to exclude spectral ranges from the correction, run this cell to indicate spectral orders or ranges to mask and then go to [ANTARESS retrieval mode](#process_data), activate `proc_data` and launch ANTARESS from the [Instrumental calibration](#inst_cal).


In [ ]:
input_nbook['sp_reduc'].update({
    'order' : [],
    'range'  : [[]]
})

ANTARESS_nbook_bground.mask_pix(input_nbook)

**Global flux balance correction**

A first correction based on the flux ratio between each exposure and its visit master computed as the median of all exposures.

- `sigma_clip` (bool): activate to remove outliers (recommended to manually exclude orders in `ord_excl_fit`)
- `nord` (float): indicate number of spectral orders, if not known set it to `None` run this cell and the cell below to plot the flux colour balance. The top axis indicates the number of spectral orders.
- `ord_excl_fit` (list): indicate which orders to be excluded from fit (not used if sigma clipping is activated)
- `phantom_bins` (float): the range of phantom bins in $\nu$ to avoid divergence in the blue part of the spectrum.
- `unc_scaling` (float): variance of fitted bins is scaled to the chosen power (0 = equal weights: 1 = no scaling (original weights), increase to give more weight to data with low errors.
- `fit_mode` (string): `'pol'` for polynomial function or `'spline'` for 1D-smoothing spline
- `pol_deg` (integer): polynomial function degree (not used in when 'spline' is chosen)
- `smooth_fac` (float): spline smoothing factor, increase to smooth

Phantom bins can be used to mirror a linear fitting onto the blue part of the spectrum. This can be used to avoid divergence at the end of the spectrum. To find the best parameters for the model an iterative approach is best used varying the different parameters.


In [ ]:
input_nbook['sp_reduc'].update({
    'sigma_clip'    : False,
    'nord'          : 170,
    'ord_excl_fit'  : [0,1,88,89,90,91,145,146,147,164,165],
    'phantom_range' : 10.,##
    'unc_scaling'   : 0.25,
    'fit_mode'      : 'spline',
#When using the polynomial function
    'pol_deg'       : 4.,
#When using the 1-D smoothing spline
    'smooth_fac'    : 1.5e-5
})

#True to calculate the correction, False to retrieve last result
input_nbook['sp_reduc'].update({
    'calc_Fbal' : False
})

ANTARESS_nbook_bground.fbal_corr(input_nbook)

**Plotting the global flux colour balance**

Plot the flux balance correction while separating the exposures (True), or plot all exposures on top of each other (Flase)

In [ ]:
input_nbook['plots'].update({
    'gap_exp': False
})

#ANTARESS_nbook_bground.plot_fbal_corr(input_nbook)

In [ ]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**Cosmics correction**

Run this cell to perform the cosmic hits correction.

The cosmic rays are detected by comparing the relative flux between exposures taken before and after. The threshold sets a limit for which a cosmic ray is considered detected. If the relative flux difference is higher than the cosmic threshold multiplied by the standard deviation of the measured or comparison spectra, a cosmic ray is considered detected.

- `align_method` (string): choose alignment method, `kep` to correct for the Keplerian curve with information entered in the system properties, `pip` to use pipeline RVs if available. Keplerian curve is preferred to avoid biased RVs due to RM effect.     
- `ncomp` (integer): number of comparison spectra to be used for the cocmic detection     
- `thresh` (float): set the threshold for which a cosmic hit is considered detected


In [ ]:
input_nbook['sp_reduc'].update({
    'calc_cosm'    : False,
    'align_method' : 'kep',
    'ncomp'        : 10,
    'thresh'       : 10.
})

ANTARESS_nbook_bground.cosm_corr(input_nbook, plot=False)

In [ ]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**ESPRESSO "wiggles" correction**

Run this cell to characterise and correct wiggles; only the screening and filter options are available in this notebook. To use the analytical version use the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In this cell, you use screening to visualise the wiggles and, from there, choose what spectral ranges in $\nu$ to include in the fitting. Generally, you will see large spurious features at the centre of the transmission spectrum located at the end and start frequencies of the red and blue detectors. Additionally, at the blue end of the spectrum, the noise levels will be much larger than the amplitude of the wiggle pattern. Hence, parts of these regions generally need to be removed from the fitting. The filter method is then used to characterise the wiggles using a Savitzky-Golay filter of the binned transmission spectrum in each exposure.

Screening:
- `screening` (bool): activate/deactivate the screening and plotting
- `fit_range` (list): indicate lower and upper boundaries of ranges to include in fit, the format is [[low 1, high 1], [low 2, high 2], [etc.]] in $\nu$, leave empty to use the full range
- `y_range` (list): list including the lower and upper y limit, if None automatic scaling is applied

Filter:
- `filter` (bool): activate/deactivate the filter correction and characterisation
- `window` (float): size of smoothing window in $\nu$
- `deg` (integer): polynomial degree used to fit the smoothed spectrum

When using the filter correction, be careful to choose an appropriate combination of window size and polynomial degree to avoid fitting noise and spurious features in the data.

In [ ]:
#Screening
input_nbook['sp_reduc'].update({
    'screening'    : False,
    'fit_range'    : [[]],
    'y_range'      : None
})

#Filter
input_nbook['sp_reduc'].update({
    'filter'       : False,
    'window'       : 0.2,
    'deg'          : 3,
})

#To calculate wiggles set to True, to retrieve previous result set to False. First time it has to be True. 
#Set corr_wig to False to deactivate the module
input_nbook['sp_reduc'].update({
    'corr_wig':False,
    'calc_wig':True
})

ANTARESS_nbook_bground.wiggle_corr(input_nbook)

In [ ]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

# Spectral processing

At this step, the original S2D spectra has been cleaned from instrumental effects, telluric contamination, cosmic ray hits, and consits of a time-series of cleaned 2D-spectra. The following steps will allow us co compute a 1D master stellar spectrum from out-of-transit data, that could be used either directly for analysis or to build/or compute a custom CCF mask.

The following steps includes procedures for aligning, scaling, and weighting the spectra to finally produce a master 1D stellar spectrum.

**Detrending of spectral lines**

If this is the first time processing the spectral time series, skip this step since the trend characteristion will be performed in a separate notebook and is performed on the CCFs computed here. After identifying any trends in the [characterisation notebook](link) you can activate the module below and define the corrections to be applied on the S2D spectra, and then recompute the CCFs from the detrended S2D specta.

To perform the detrending in this step, the [characterisation notebook](link) must have been used to determine any trends in the data and appropriate corrections first. Here you will use the values for the coefficients determined using the [characterisation notebook](link). Define which proeprty to correct and the corresponding dependence. If multiple properties are being corrected add them to the list.

- `use` (bool): set to True to perform a trend correction
- `prop` (list, str): list each property to correct with the correcsponding dependece. Properties to correct here is either RV or contrast (ctrst), and as a function of either phase or snr (with ESPRESSO snrQ)
- `coeff` (list,list): indicate coefficients used for each model, add coefficients with decreasing order, add a new list if more than one property is corrected. The RV coefficient should be in km/s.


In [ ]:
input_nbook['par'].update({
    'use'  : False,
    'prop' : ['ctrst_snrQ', 'RV_phase'],
    'mode' : ['pol', 'pol'],
    'coeff': [[-1.109190e-05], [-6.840494e-02]]
})

ANTARESS_nbook_bground.detrend(input_nbook)

**Compute CCF from spectra**

Comupute CCF from input spectra using used defined CCF masks.

 + `type` (string): what data type, `DI` or `Intr`
 + `start_RV` (float): start RV for CCF computation in km/s, in stellar restframe
 + `end_RV` (float): end RV for CCF computation in km/s, in stellar restframe
 + `dRV` (float): step size in RV, use `None` for instrumental resolution. Using smaller steps than resolution will introduce correlations.
 + `mask_path` (string): location where the mask is stored + filename (relative to `'working_path'`)


In [ ]:
input_nbook['par'].update({
    'type'     : 'DI',
    'start_RV' : -150.,
    'end_RV'   : 150.,
    'dRV'      : None,
    'mask_path': 'CCF_masks/ESPRESSO_new_G9.fits',
    'calc_CCF' : True
})

ANTARESS_nbook_bground.conv_CCF(input_nbook)

In [ ]:
ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

In [ ]:
%tb